# Arps Decline Curve Analysis - CSV Example

This notebook demonstrates how to perform Arps decline curve analysis using CSV files instead of a SQL database.

In [ ]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path().absolute().parent.parent
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from AnalyticsAndDBScripts.csv_loader import CSVDataLoader
import AnalyticsAndDBScripts.prod_fcst_functions as fcst

print("✅ Imports successful!")

## Step 1: Create Sample Data

First, let's create sample CSV files for testing:

In [ ]:
from AnalyticsAndDBScripts.csv_loader import create_sample_csv_files

# Create sample files in project root
prod_file, well_file = create_sample_csv_files(output_dir=str(project_root))

print(f"Created:\n  {prod_file}\n  {well_file}")

## Step 2: Load Data

Initialize the CSV loader and load production data:

In [ ]:
# Initialize CSV loader
csv_loader = CSVDataLoader(
    production_csv_path=str(prod_file),
    well_list_csv_path=str(well_file)
)

# Load data
prod_df = csv_loader.load_production_data()
well_list = csv_loader.load_well_list()

print(f"\nLoaded {len(prod_df)} production records")
print(f"Wells to analyze: {len(well_list)}")

## Step 3: Explore the Data

In [ ]:
# Display summary statistics
stats = csv_loader.get_summary_stats()
print("\nSummary Statistics:")
for key, value in stats.items():
    print(f"  {key}: {value}")

# Show first few rows
print("\nSample Production Data:")
display(prod_df.head(10))

## Step 4: Visualize Production Data

Plot production history for a specific well:

In [ ]:
# Select a well to visualize
wellid = well_list['WellID'].iloc[0]

# Get production data for this well
well_prod = prod_df[prod_df['WellID'] == wellid]

# Create plot
fig, axes = plt.subplots(3, 1, figsize=(12, 10))
fig.suptitle(f'Production History - Well {wellid}', fontsize=16)

for idx, (measure, ax) in enumerate(zip(['OIL', 'GAS', 'WATER'], axes)):
    data = well_prod[well_prod['Measure'] == measure]
    ax.plot(data['Date'], data['Value'], 'o-', label=measure)
    ax.set_ylabel(f'{measure} Rate')
    ax.set_xlabel('Date')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Step 5: Run Arps Analysis

Use the command-line script to fit Arps curves:

In [ ]:
# Run the CSV-based Arps fitting script
import subprocess

result = subprocess.run([
    'python',
    str(project_root / 'play_assesments_tools' / 'python files' / 'arps_autofit_csv.py'),
    str(prod_file),
    '--well-list', str(well_file),
    '--output', str(project_root / 'arps_results.csv')
], capture_output=True, text=True)

print(result.stdout)
if result.returncode != 0:
    print("Error:", result.stderr)

## Step 6: View Results

In [ ]:
# Load results
results_df = pd.read_csv(project_root / 'arps_results.csv')

print(f"\nFitted {len(results_df)} wells")
print(f"Average R²: {results_df['R_squared'].mean():.3f}")
print(f"\nResults:")
display(results_df)

## Step 7: Visualize Fitted Curves

Plot actual vs. fitted production:

In [ ]:
# Select a well/measure to visualize
result_row = results_df.iloc[0]
wellid = int(result_row['WellID'])
measure = result_row['Measure']

# Get actual production data
actual_data = csv_loader.get_well_production(
    wellid=wellid,
    measure=measure,
    last_prod_date=pd.Timestamp('2024-10-01'),
    fit_months=60
)

# IMPORTANT: Generate forecast with same time array as fitting
# The forecast should start at t=0 corresponding to the first actual data point
t_months = np.arange(0, len(actual_data) + 12)
forecast = fcst.varps_decline(
    wellid, 1,
    result_row['Q3'],  # Fitted Qi
    result_row['Dei'],
    0.08 if measure == 'OIL' else 0.06,  # Def
    result_row['b_factor'],
    t_months, 0, 0
)

# Extract forecast rates
forecast_rates = forecast[3]

# VALIDATION CHECK: Verify first point alignment
first_actual = actual_data['Value'].iloc[0]
first_forecast = forecast_rates[0]
error_pct = abs(first_forecast - first_actual) / first_actual * 100

print(f"⚠️  VALIDATION CHECK:")
print(f"  First Actual Rate: {first_actual:.2f}")
print(f"  First Forecast Rate (q(0)): {first_forecast:.2f}")
print(f"  Error: {error_pct:.1f}%")
if error_pct > 15:
    print(f"  ❌ WARNING: First point mismatch > 15%!")
else:
    print(f"  ✓ First point alignment acceptable")

# Plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Create forecast dates
start_date = actual_data['Date'].min()
forecast_dates = pd.date_range(start=start_date, periods=len(t_months), freq='MS')

# Top plot: Full view with forecast
ax1.plot(actual_data['Date'], actual_data['Value'], 'o', 
         label='Actual Production', markersize=8, color='#1f77b4', zorder=3)
ax1.plot(forecast_dates[:len(actual_data)], forecast_rates[:len(actual_data)], 
         '-', label='Arps Fit (History)', linewidth=2.5, color='#d62728', zorder=2)
ax1.plot(forecast_dates[len(actual_data):], forecast_rates[len(actual_data):], 
         '--', label='Arps Forecast (Future)', linewidth=2.5, color='#ff7f0e', zorder=2)

ax1.set_xlabel('Date', fontsize=11)
ax1.set_ylabel(f'{measure} Rate (Bbl/day or Mcf/day)', fontsize=11)
ax1.set_title(f'Well {wellid} - {measure} Decline Curve\nR² = {result_row["R_squared"]:.3f}, '
              f'First Point Error = {error_pct:.1f}%', fontsize=13, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Bottom plot: Residuals
residuals = actual_data['Value'].values - forecast_rates[:len(actual_data)]
ax2.bar(actual_data['Date'], residuals, width=20, 
        color=['red' if r < 0 else 'green' for r in residuals], alpha=0.6)
ax2.axhline(y=0, color='black', linestyle='-', linewidth=1)
ax2.set_xlabel('Date', fontsize=11)
ax2.set_ylabel('Residual (Actual - Predicted)', fontsize=11)
ax2.set_title('Residual Analysis', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"\n📊 Fitted Parameters:")
print(f"  Qi (Initial Rate): {result_row['Q3']:.2f}")
print(f"  Dei (Initial Decline): {result_row['Dei']:.4f} ({result_row['Dei']*100:.2f}%/year)")
print(f"  b-factor: {result_row['b_factor']:.3f}")
print(f"  R²: {result_row['R_squared']:.3f}")
print(f"  RMSE: {result_row['RMSE']:.2f}")
print(f"  MAE: {result_row['MAE']:.2f}")

## Next Steps

- Replace sample data with your own CSV files
- Adjust configuration parameters in `config/analytics_config.yaml`
- Export forecasts for economic analysis
- Use results in Streamlit app for interactive visualization